In [5]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from skrub import TableReport

import os, glob
import gc

In [2]:
COMP_DATA_BASE = os.path.join("..", "data", "comp")
PREP_DATA_BASE = os.path.join("..", "data", "preprocessed")

TRAIN_PATH = os.path.join(COMP_DATA_BASE, "train.csv")
TEST_PATH = os.path.join(COMP_DATA_BASE, "test.csv")
DATA_DICT_PATH = os.path.join(COMP_DATA_BASE, "data_dictionary.csv")
SAMPLE_PATH = os.path.join(COMP_DATA_BASE, "sample_submission.csv")

features = list(pl.read_csv(TEST_PATH).select(pl.all().exclude("ID")).columns)
train_ds = pl.read_csv(TRAIN_PATH)

In [3]:
num_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Numerical"))
num_features = num_features_dict.select("variable").to_series().to_list()
cat_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Categorical"))
cat_features = cat_features_dict.select("variable").to_series().to_list()
misc_columns = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features).not_())
display(num_features_dict)
display(cat_features_dict)
display(misc_columns)


variable,description,type,values
str,str,str,str
"""hla_match_c_high""","""Recipient / 1st donor allele l…","""Numerical""",null
"""hla_high_res_8""","""Recipient / 1st donor allele-l…","""Numerical""",null
"""hla_low_res_6""","""Recipient / 1st donor antigen-…","""Numerical""",null
"""hla_high_res_6""","""Recipient / 1st donor allele-l…","""Numerical""",null
"""hla_high_res_10""","""Recipient / 1st donor allele-l…","""Numerical""",null
…,…,…,…
"""comorbidity_score""","""Sorror comorbidity score""","""Numerical""",null
"""karnofsky_score""","""KPS at HCT""","""Numerical""",null
"""hla_low_res_8""","""Recipient / 1st donor antigen-…","""Numerical""",null


variable,description,type,values
str,str,str,str
"""dri_score""","""Refined disease risk index""","""Categorical""","""['Intermediate' 'High' 'N/A - …"
"""psych_disturb""","""Psychiatric disturbance""","""Categorical""","""['Yes' 'No' nan 'Not done']"""
"""cyto_score""","""Cytogenetic score""","""Categorical""","""['Intermediate' 'Favorable' 'P…"
"""diabetes""","""Diabetes""","""Categorical""","""['No' 'Yes' nan 'Not done']"""
"""tbi_status""","""TBI""","""Categorical""","""['No TBI' 'TBI + Cy +- Other' …"
…,…,…,…
"""tce_div_match""","""T-cell epitope matching""","""Categorical""","""['Permissive mismatched' 'Bi-d…"
"""donor_related""","""Related vs. unrelated donor""","""Categorical""","""['Unrelated' 'Related' 'Multip…"
"""melphalan_dose""","""Melphalan dose (mg/m^2)""","""Categorical""","""['N/A, Mel not given' 'MEL' na…"


variable,description,type,values
str,str,str,str
"""efs""","""Event-free survival""","""Categorical""","""['Event' 'Censoring']"""
"""efs_time""","""Time to event-free survival, m…","""Numerical""",null


In [6]:
my_tablereport = TableReport(train_ds)
my_tablereport

Processing column  60 / 60


<TableReport: use .open() to display>